In [1]:
import copy

# 外皮情報LV2からLV3へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）
- 住戸全体の外皮（床下空間に属するものを除く）、および、室と床下空間の間の床は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。
- 各部位の熱橋は、対応する一般部のU値の低減に置き換える。一般部の断熱材の厚を調整する。
- その他は、用途別情報が得られている場合と同様に求める。

### 《土間床等面積が0㎡より大きい場合の扱い》
$$ \qquad
A_{b}=min(A_主 + A_他 + A_非, \sum_{i}^{}S_{df,i})
\qquad
\\
$$

- 入力値：
  - 室用途不明の空間に属する土間床等の部位iの面積$[m ^ 2]$：$S_{df,i}$

- 出力値：

  - 室と床下空間の間の床面積の合計$[m ^ 2]$：$A_{b}$

### 1) 室用途別面積比率

#### Outline

$$ 
r_m = \frac{ a_m }{ A_m + A_o + A_n }
$$

$$
r_o = \frac{ A_o }{ A_m + A_o + A_n }
$$

$$
r_n = \frac{ A_n }{ A_m + A_o + A_n }
$$

$r_m$: 主たる居室の面積比率, -  
$r_o$: その他の居室の面積比率, -  
$r_n$: 非居室の面積比率, -  
$A_m$: 主たる居室の面積, m<sup>2</sup>  
$A_o$: その他の居室の面積, m<sup>2</sup>  
$A_n$: 非居室の面積, m<sup>2</sup>  

#### Function

In [2]:
def get_area_rate(a_m, a_o, a_t):
    # a_m is the main occupied floor area as float, m2
    # a_o is the other occupied floor area as float, m2
    # a_t is the total occupied floor area as float, m2
    
    # a_n is the non ocupied floor area as float, m2
    a_n = a_t - a_m - a_o

    return a_m / a_t, a_o / a_t, a_n / a_t

### 2) 室用途別の面積の推定

#### Outline

$$
A_{m,i}= A_{i} \times r_m
$$

$$
A_{o,i}= A_{i} \times r_o
$$

$$
A_{n,i}= A_{i} \times r_n
$$

$A_{m,i}$: 主たる居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{o,i}$: その他の居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{n,i}$: 非居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_i$: 外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>

#### Function

In [3]:
def get_area_by_room_use( target_area, a_m, a_o, a_t ):
    # target_area is the area of the part i as double, m2
    # a_m is the main floor area as float, m2
    # a_o is the other floor area as float, m2
    # a_t is the total floor area as float, m2
    
    r_m, r_o, r_n = get_area_rate( a_m, a_o, a_t )
    return {
        'main'      : r_m * target_area,
        'other'     : r_o * target_area,
        'nonliving' : r_n * target_area
    }

### 3) 室用途別の長さの推定

#### Outline

$$
L_{m,i}= L_{i} \times r_m
$$

$$
L_{o,i}= L_{i} \times r_o
$$

$$
L_{n,i}= L_{i} \times r_n
$$

$L_{m,i}$: 主たる居室の熱橋及び土間床等の外周部$i$の長さ, m  
$L_{o,i}$: その他の熱橋及び土間床等の外周部$i$の長さ, m  
$L_{n,i}$: 非居室の熱橋及び土間床等の外周部$i$の長さ, m  
$L_i$: 熱橋及び土間床等の外周部$i$の長さ, m

#### Function

In [4]:
def get_length_by_room_use(target_length, a_m, a_o, a_t):
    # target_length is the length of the part i as double, m
    # a_m is the main floor area as float, m
    # a_o is the other floor area as float, m
    # a_t is the total floor area as float, m
    
    r_m, r_o, r_n = get_area_rate(a_m, a_o, a_t)
    return {
        'main'      : r_m * target_length,
        'other'     : r_o * target_length,
        'nonliving' : r_n * target_length
    }

#### Example

In [5]:
areas = get_area_by_room_use(50.0, 30.0, 60.0, 120.0)
lengths = get_length_by_room_use(12.0, 30.0, 60.0, 120.0)
areas, lengths

({'main': 12.5, 'nonliving': 12.5, 'other': 25.0},
 {'main': 3.0, 'nonliving': 3.0, 'other': 6.0})

## 2. Convert

### 1) 共通

'Common'要素はそのまま維持する。

#### Input

参照：InputDataDefinition.ipynb

#### Output

参照：InputDataDefinition.ipynb

#### Function

In [6]:
def convert_common(d):
    return d['Common']

#### Example

In [7]:
d = {
    'Common': {
        'Region' : 6,
        'IsSimplifiedInput': True,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    }
}

convert_common(d)

{'IsSimplifiedInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'Region': 6,
 'TotalFloorArea': 120.0}

### 2) 開口部以外の不透明部位

- 壁体名称、一般部位構造種別、壁体種別、方位、層構成（部材名称、厚さ[m]、熱伝導率[W/mK]、容積比熱[(J/(m<sup>3</sup>・K)]）などの情報は維持する。
- Lv2の入力情報に「壁体が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [8]:
def make_wall(name, direction, area, space):
    return {
        'name':      name + '_' + space,
        'direction': direction,
        'area':      area,
        'space':     space
    }

In [9]:
def convert_wall( d ):
    walls =[]
    for arg_wall in d['Walls']:
        areas = get_area_by_room_use(
            target_area=arg_wall['area'], 
            a_m = d['Common']['MainOccupantRoomFloorArea'], 
            a_o = d['Common']['OtherOccupantRoomFloorArea'], 
            a_t = d['Common']['TotalFloorArea']
        )
        for space, area in areas.items() :
             # 壁体名称、部位面積を更新し、「壁体が属する空間種類」を追加する
            wall = make_wall(
                name      = arg_wall['name'], 
                direction = arg_wall['direction'], 
                area      = area, 
                space     = space
            )
            walls.append(wall)
            
    return walls

#### Example

In [10]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Walls': [
        { 'area': 67.8, 'direction': 'top', 'name': 'Ceiling' },
        { 'area': 40.63, 'direction': 'SW', 'name': 'Wall_SW',}
    ]
}    

convert_wall(d)

[{'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main'},
 {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other'},
 {'area': 16.95,
  'direction': 'top',
  'name': 'Ceiling_nonliving',
  'space': 'nonliving'},
 {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main'},
 {'area': 20.315,
  'direction': 'SW',
  'name': 'Wall_SW_other',
  'space': 'other'},
 {'area': 10.1575,
  'direction': 'SW',
  'name': 'Wall_SW_nonliving',
  'space': 'nonliving'}]

### 3) 開口部の透明部位

- 開口部名称、方位、日射熱取得率[-]、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [11]:
def make_window(name, direction, area, space):
    return {
        'name':      name + '_' + space,
        'direction': direction,
        'area':      area,
        'space':     space
    }

In [12]:
def convert_window( d ):
    
    windows =[]
    for arg_window in d['Windows']:
        areas = get_area_by_room_use(
            target_area = arg_window['area'], 
            a_m = d['Common']['MainOccupantRoomFloorArea'], 
            a_o = d['Common']['OtherOccupantRoomFloorArea'], 
            a_t = d['Common']['TotalFloorArea']
        )
        for space, area in areas.items() :
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            window = make_window(
                name = arg_window['name'],
                direction = arg_window['direction'],
                area = area,
                space = space
            )
            windows.append(window)
        
    return windows

#### Example

In [13]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17  }
    ]
}    

convert_window(d)

[{'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_main', 'space': 'main'},
 {'area': 15.125,
  'direction': 'SW',
  'name': 'WindowSW_other',
  'space': 'other'},
 {'area': 7.5625,
  'direction': 'SW',
  'name': 'WindowSW_nonliving',
  'space': 'nonliving'},
 {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_main', 'space': 'main'},
 {'area': 1.585,
  'direction': 'NW',
  'name': 'WindowNW_other',
  'space': 'other'},
 {'area': 0.7925,
  'direction': 'NW',
  'name': 'WindowNW_nonliving',
  'space': 'nonliving'}]

### 4) 開口部の不透明部位

- 開口部名称、方位、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [14]:
def make_door(name, direction, area, space):
    return {
        'name':      name + '_' + space,
        'direction': direction,
        'area':      area,
        'space':     space
    }

In [15]:
def convert_door( d ):
    
    doors =[]
    for arg_door in d['Doors']:
        areas = get_area_by_room_use(arg_door['area'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, area in areas.items() :
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            door = make_door(
                name      = arg_door['name'],
                direction = arg_door['direction'],
                area      = area,
                space     = space
            )
            doors.append(door)
        
    return doors

#### Example

In [16]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 52 }
    ]
}    
convert_door(d)

[{'area': 13.0, 'direction': 'NW', 'name': 'DoorNW_main', 'space': 'main'},
 {'area': 26.0, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
 {'area': 13.0,
  'direction': 'NW',
  'name': 'DoorNW_nonliving',
  'space': 'nonliving'}]

### 5) 線熱橋

- 熱橋部の構造種別、方位、線熱貫流率[W/(m・K)]などの情報は維持する。
- 「 熱橋部が属する空間種類」の情報を追加し、熱橋長さから室用途別の熱橋長さを推定する。

#### Function

In [17]:
def make_heatbridge(structure, direction1, direction2, length, space):
    return {
        'structure':  structure,
        'direction1': direction1,
        'direction2': direction2,
        'length':     length,
        'space':      space
    }

In [18]:
def convert_heatbridge( d ):
    
    heatbridges =[]
    for arg_heatbridge in d['Heatbridges']:
        lengths = get_length_by_room_use(arg_heatbridge['length'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, length in lengths.items() :
            # 熱橋部長さを更新し、「熱橋部が属する空間種類」を追加する
            heatbridge = make_heatbridge(
                structure  = arg_heatbridge['structure'],
                direction1 = arg_heatbridge['direction1'],
                direction2 = arg_heatbridge['direction2'],
                length     = length,
                space      = space,
            )
            heatbridges.append(heatbridge)
        
    return heatbridges

#### Example

In [19]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Heatbridges': [
        { 'structure': 'rc', 'direction1': 'NW', 'direction2': 'NE', 'length': 10 }
    ]
}    
convert_heatbridge(d)

[{'direction1': 'NW',
  'direction2': 'NE',
  'length': 2.5,
  'space': 'main',
  'structure': 'rc'},
 {'direction1': 'NW',
  'direction2': 'NE',
  'length': 5.0,
  'space': 'other',
  'structure': 'rc'},
 {'direction1': 'NW',
  'direction2': 'NE',
  'length': 2.5,
  'space': 'nonliving',
  'structure': 'rc'}]

### 6) 土間床等の外周部の線熱橋

- 方位、熱橋長さ[m]、線熱貫流率[W/(m・K)]などの情報は維持する。
- 「土間床等の外周部の線熱橋が属する空間種類」の情報を追加する。ただし、Lv2からLv3への変換においては、空間種類は「床下空間」のみとする。

#### Function

In [20]:
def make_earthfloorperimeter( direction, length, name, psi):
    return {
        'direction': direction,
        'length':    length,
        'name':      name,
        'psi':       psi,
        'space':     'underfloor',
    }

In [21]:
def convert_earthfloorperimeter( d ):    
    return [ make_earthfloorperimeter(**f) for f in d['EarthfloorPerimeters'] ]

#### Example

In [22]:
d = {
    'EarthfloorPerimeters': [
        { 'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8 },
        { 'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8 }
    ]
}    
convert_earthfloorperimeter(d)

[{'direction': 'NW',
  'length': 2.43,
  'name': 'other_NW',
  'psi': 1.8,
  'space': 'underfloor'},
 {'direction': 'NE',
  'length': 2.43,
  'name': 'other_NE',
  'psi': 1.8,
  'space': 'underfloor'}]

### 7) 土間床
- 面積[m<sup>2</sup>]などの情報は維持する。
- 「土間床等が属する空間種類」の情報を追加する。ただし、Lv2からLv3への変換においては、空間種類は「床下空間」のみとする。

#### Function

In [23]:
def make_earthfloor( name, area ):
    return {
        'name': name,
        'area': area,
        'space': 'underfloor'
    }
    return earthfloor

In [24]:
def convert_earthfloor( d ):        
    return [ make_earthfloor(**f) for f in d['Earthfloors'] ]

#### Example

In [25]:
d = {
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0 }
    ]
}    
convert_earthfloor(d)

[{'area': 5.0, 'name': 'other', 'space': 'underfloor'}]

## 3. Lv3入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床の各コンバート関数を統合し、Lv2の入力情報をLv3の入力情報にコンバートする。

### Function

In [26]:
def convert(d):
    d_lv3_input = {}
    d_lv3_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv3_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv3_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv3_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv3_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('Earthfloors' in d) == True:
        d_lv3_input['Earthfloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv3_input['Heatbridges'] = convert_heatbridge(d)

    return d_lv3_input

### Example

In [27]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': 2,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'wood', 'InputMethodWood' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'wood', 'InputMethodWood' :'InputAllDetails', 'direction': 'top', 'area': 67.8,
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'floor', 'structure': 'wood', 'InputMethodWood' :'InputAllLayers', 'direction': 'bottom', 'area': 67.8,
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False },
        { 'name': 'Wall', 'type': 'wall', 'structure': 'wood', 'InputMethodWood' :'InputAllLayers', 'direction': 'N', 'area': 67.8, 
          'WallConstructionMethod': 'WallInsuladdBackvertical',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'wood', 'InputMethodWood' :'InputAllLayers', 'direction': 'top', 'area': 67.8,
          'RoofConstructionMethod': 'Insulrafter', 
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'BoundaryCeiling', 'type': 'boundary_ceiling', 'structure': 'wood', 'InputMethodWood' :'InputAllLayers', 
          'direction': 'top', 'area': 67.8, 'CeilingConstructionMethod': 'Insulbeam', 
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'wood', 'InputMethodWood' :'InputUR',
          'direction': 'top', 'area': 67.8, 'URWood': 0.05, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'RC', 'InputMethodRC' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'RC', 'InputMethodRC' :'InputLayers', 
          'direction': 'top', 'area': 67.8, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'steel', 'InputMethodSteel' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'steel', 'InputMethodSteel' :'InputUR', 
          'direction': 'top', 'area': 67.8, 'URSteel': 0.10, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'other', 'direction': 'top', 'area': 67.8, 'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'UW': 6.51, 'type': 'single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'UW': 4.65, 'type': 'single', 
          'IsEtaValueInput': True, 'Eta': 0.738, 'IsSunshadeInput': False },
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'UW': 6.51, 'type': 'double', 'IsEtaValueInput': False,
          'TypeFrameInside': 'WoodOrResin', 'TypeGlassInside': '3WgG', 'TypeShadeInside': 'Shoji',
          'TypeFrameOutside': 'Steel', 'TypeGlassOutside': '3WgG', 'TypeShadeOutside': 'ExtarnalBlind',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'UW': 4.65, 'type': 'double', 'IsEtaValueInput': True,
          'EtaInside': 0.738, 'TypeGlassInside': '3WgG', 'EtaOutside': 0.738, 'TypeGlassOutside': '3WgG',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 2.52, 'U': 6.51, 'IsSunshadeInput': False },
        { 'name': 'DoorNE', 'direction': 'NE', 'area': 2.16, 'U': 4.65, 'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Heatbridges': [
        { 'name': 'NE', 'structure': 'RC', 'length': 1.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'E' },
        { 'name': 'NW', 'structure': 'Steel', 'length': 2.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'W' }
    ],
    'EarthfloorPerimeters': [
        { 'name': 'other_NW', 'direction': 'NW', 'length': 2.43, 'psi': 1.8 },
        { 'name': 'other_NE', 'direction': 'NE', 'length': 2.43, 'psi': 1.8 }
    ],
    'Earthfloors': [
        { 'area': 5.0, 'name': 'other' },
        { 'area': 5.0, 'name': 'entrance' }
    ]
}

convert(d)

{'Common': {'IsSimplifiedInput': 2,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'Doors': [{'area': 0.63,
   'direction': 'NW',
   'name': 'DoorNW_main',
   'space': 'main'},
  {'area': 1.26, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
  {'area': 0.63,
   'direction': 'NW',
   'name': 'DoorNW_nonliving',
   'space': 'nonliving'},
  {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_main', 'space': 'main'},
  {'area': 1.08, 'direction': 'NE', 'name': 'DoorNE_other', 'space': 'other'},
  {'area': 0.54,
   'direction': 'NE',
   'name': 'DoorNE_nonliving',
   'space': 'nonliving'}],
 'EarthfloorPerimeters': [{'direction': 'NW',
   'length': 2.43,
   'name': 'other_NW',
   'psi': 1.8,
   'space': 'underfloor'},
  {'direction': 'NE',
   'length': 2.43,
   'name': 'other_NE',
   'psi': 1.8,
   'space': 'underfloor'}],
 'Earthfloors': [{'area': 5.0, 'name': 'other', 'space': 'underfloor'},
  {'area': 5.0